In [1]:
import time
notebook_start = time.time()

In [2]:
# Cell 1: Complete Environment Setup
# ========================================================

# 1. First, clean up everything - more thorough cleanup
import sys
import shutil
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["OMP_NUM_THREADS"] = "1"
import json

# Clean up problematic installations
!pip uninstall -y numpy torch torchvision torchaudio transformers peft bitsandbytes 2>/dev/null || echo "No packages to uninstall"

# Remove problematic directories manually
problematic_path = "/usr/local/lib/python3.11/dist-packages/~vidia-cudnn-cu12"
if os.path.exists(problematic_path):
    shutil.rmtree(problematic_path)

# Clear pip cache
!pip cache purge

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: torch 2.2.1+cu121
Uninstalling torch-2.2.1+cu121:
  Successfully uninstalled torch-2.2.1+cu121
Found existing installation: torchvision 0.17.1+cu121
Uninstalling torchvision-0.17.1+cu121:
  Successfully uninstalled torchvision-0.17.1+cu121
Found existing installation: torchaudio 2.2.1+cu121
Uninstalling torchaudio-2.2.1+cu121:
  Successfully uninstalled torchaudio-2.2.1+cu121
Found existing installation: transformers 4.41.2
Uninstalling transformers-4.41.2:
  Successfully uninstalled transformers-4.41.2
Found existing installation: peft 0.10.0
Uninstalling peft-0.10.0:
  Successfully uninstalled peft-0.10.0
Found existing installation: bitsandbytes 0.43.0
Uninstalling bitsandbytes-0.43.0:
  Successfully uninstalled bitsandbytes-0.43.0
Files removed: 138


In [3]:
pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 71.2 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 0.29.1 requires torch>=1.10.0, which is not installed.
easyocr 1.7.2 requires torch, which is not installed.
easyocr 1.7.2 requires torchvision>=0.5, which is not installed.
torchmetrics 1.7.1 requires torch>=2.0.0, which is not installed.
pytorch-lightning 2.5.1.post0 requires torch>=2.1.0, which is not installed.
kaggle-environments 1.16.11 requires transformers>=4.33.1, which is not installed.
stable-baselines3 2.1.0 requires torch>=1.13, which is not installed.
sentence-transformers 3.4.1 requires torch>=1.11.0, which is not installed.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, which is not installed.
fastai 2

In [19]:
# Create a virtual environment to isolate our dependencies
!python -m venv myenv
!source myenv/bin/activate

# Upgrade pip and setuptools first
!pip install --upgrade pip setuptools wheel

# Install core numerical packages with pinned versions
!pip install numpy==1.26.4 scipy==1.11.4

# Install PyTorch CPU version
!pip install torch==2.2.1+cpu torchvision==0.17.1+cpu torchaudio==2.2.1+cpu --index-url https://download.pytorch.org/whl/cpu

# Install transformer ecosystem with compatible versions
!pip install \
    transformers==4.41.2 \
    peft==0.10.0 \
    datasets==2.18.0 \
    accelerate==0.29.1 \
    einops==0.7.0 \
    tokenizers==0.19.1 \
    sentencepiece==0.2.0

# Install required ML utilities
!pip install scikit-learn==1.2.2 matplotlib==3.7.2

# Install project-specific dependencies
!pip install \
    langchain==0.1.16 \
    faiss-cpu==1.7.4 \
    tqdm==4.66.2 \
    pandas==2.2.2 \
    google-auth==2.38.0

# Verify installations
import sys
import numpy as np
import scipy
import torch
import transformers

print("\n=== Core Package Versions ===")
print(f"Python: {sys.version}")
print(f"NumPy: {np.__version__}")
print(f"SciPy: {scipy.__version__}")
print(f"PyTorch: {torch.__version__}")
print(f"Transformers: {transformers.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# Now import your project packages
from datasets import Dataset, load_dataset, DatasetDict
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling
)
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import re

print("\n=== All Packages Imported Successfully ===")

Error: Command '['/kaggle/working/myenv/bin/python3', '-m', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
/bin/bash: line 1: myenv/bin/activate: No such file or directory
Looking in indexes: https://download.pytorch.org/whl/cpu

=== Core Package Versions ===
Python: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
NumPy: 1.26.4
SciPy: 1.15.2
PyTorch: 2.2.1+cu121
Transformers: 4.41.2
CUDA available: False

=== All Packages Imported Successfully ===


In [20]:
# Model Loading
# Define MODEL_NAME at the top of the cell (should match what is used in Cell 1)
MODEL_NAME = "gpt2"  # Change to "meta-llama/Llama-2-7b-chat-hf" for Llama

def print_memory():
    """Memory usage diagnostics for the environment"""
    ram = psutil.virtual_memory()
    print(f"RAM: {ram.percent:.1f}% ({ram.used/1024**3:.1f}/{ram.total/1024**3:.1f}GB)")

def load_model(model_name):
    print(f"\n=== Loading Model: {model_name} ===")
    print_memory()
    
    # CPU-specific settings
    device = "cpu"
    torch_dtype = torch.float32
    trust_remote_code = "phi" in model_name.lower()
    
    try:
        print("Attempting standard CPU load...")
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=trust_remote_code,
            device_map=None,
            torch_dtype=torch_dtype
        ).to(device)
        print("\n✅ Model loaded successfully on CPU!")
        return model
    except Exception as e:
        print(f"\n❌ Standard load failed: {str(e)}")
        raise RuntimeError("Unable to load model on CPU")

model = load_model(MODEL_NAME)


=== Loading Model: gpt2 ===
RAM: 7.0% (1.8/31.4GB)
Attempting standard CPU load...

✅ Model loaded successfully on CPU!


In [21]:
# 4. Tokenizer Setup
def load_tokenizer(model_name):
    try:
        tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=True,
            padding_side="right"
        )
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        print("Tokenizer loaded successfully")
        return tokenizer
    except Exception as e:
        print(f"Tokenizer loading failed: {str(e)}")
        raise

tokenizer = load_tokenizer(MODEL_NAME)

Tokenizer loaded successfully


In [22]:
# Robust Data Preparation
# 1. Dataset preparation with multiple fallbacks
def prepare_dataset(file_path="/kaggle/input/database4", max_samples=1000):
    try:
        # Check if path is a directory or file
        if os.path.isdir(file_path):
            # Directory - look for JSON files
            json_files = [f for f in os.listdir(file_path) if f.endswith('.json')]
            if not json_files:
                raise FileNotFoundError(f"No JSON files found in directory: {file_path}")
            data_files = [os.path.join(file_path, f) for f in json_files]
            print(f"Found {len(json_files)} JSON files in directory")
        elif os.path.isfile(file_path):
            # Single file
            data_files = file_path
        else:
            raise FileNotFoundError(f"Path not found: {file_path}")
            
        # Load dataset
        dataset = load_dataset('json', data_files=data_files, split=f'train[:{max_samples}]')
        
        # Ensure text column exists
        if 'text' not in dataset.features:
            text_columns = [col for col in dataset.features if 'text' in col.lower()]
            if text_columns:
                dataset = dataset.rename_column(text_columns[0], 'text')
            else:
                string_columns = [col for col in dataset.features if dataset.features[col].dtype == 'string']
                if string_columns:
                    def combine_columns(examples):
                        return {'text': ' '.join(str(examples[col]) for col in string_columns)}
                    dataset = dataset.map(combine_columns)
                else:
                    raise ValueError("No text columns found in dataset")
        
        print(f"✅ Loaded dataset with {len(dataset)} samples")
        return dataset
    except Exception as e:
        print(f"\n❌ Dataset preparation failed: {str(e)}")
        print("Creating minimal fallback dataset...")
        return Dataset.from_dict({"text": [f"Sample text {i}" for i in range(10)]})

# 2. Tokenization function
def safe_tokenize(examples):
    try:
        tokenized = tokenizer(
            examples["text"],
            truncation=True,
            max_length=160,
            padding="max_length",
            return_tensors="pt"
        )
        return {
            "input_ids": tokenized["input_ids"].tolist(),
            "attention_mask": tokenized["attention_mask"].tolist(),
            "labels": tokenized["input_ids"].tolist()
        }
    except Exception as e:
        print(f"Tokenization error: {str(e)}")
        return {
            "input_ids": [[0]*160],
            "attention_mask": [[1]*160],
            "labels": [[0]*160]
        }

# 3. Process dataset
print("\n=== Starting Data Processing ===")
dataset = prepare_dataset()

# Tokenize dataset
tokenized_dataset = dataset.map(safe_tokenize, batched=True, batch_size=4)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Split dataset
if len(tokenized_dataset) > 1:
    split_datasets = tokenized_dataset.train_test_split(test_size=0.1)
    tokenized_dataset = DatasetDict({
        "train": split_datasets["train"],
        "test": split_datasets["test"]
    })
    print(f"✅ Dataset split: {len(tokenized_dataset['train'])} train, {len(tokenized_dataset['test'])} test")
else:
    # Fallback if not enough samples
    tokenized_dataset = DatasetDict({
        "train": tokenized_dataset,
        "test": Dataset.from_dict({
            "input_ids": [[0,1,2,3]],
            "attention_mask": [[1,1,1,1]],
            "labels": [[0,1,2,3]]
        })
    })
    print("⚠️ Using minimal test set")


=== Starting Data Processing ===

❌ Dataset preparation failed: No JSON files found in directory: /kaggle/input/database4
Creating minimal fallback dataset...


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

✅ Dataset split: 9 train, 1 test


In [23]:
# Training Configuration
# Enable gradient checkpointing to save memory
model.gradient_checkpointing_enable()

# LoRA configuration
from peft import LoraConfig

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["attn.c_attn", "attn.c_proj", "mlp.c_fc", "mlp.c_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    fan_in_fan_out=False
)

# Training arguments optimized for Kaggle
training_args = TrainingArguments(
    output_dir=f"./{MODEL_NAME.split('/')[-1]}-crypto-expert",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-5,
    optim="adamw_torch",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    fp16=False,  # Disable for CPU
    bf16=False,  # Disable for CPU
    max_grad_norm=0.3,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    no_cuda=True  # Explicitly disable CUDA
)

# 3. Prepare model
if hasattr(model, "gradient_checkpointing_enable"):
    model.gradient_checkpointing_enable()
    
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# 4. Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

trainable params: 2,359,296 || all params: 126,799,104 || trainable%: 1.8606566809809635


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1489: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [24]:
!pip install memory_profiler
from memory_profiler import profile

# 2. Tokenization function with memory optimizations
def safe_tokenize(examples):
    """Tokenization with explicit numpy workarounds and memory efficiency"""
    try:
        # Process text in chunks to avoid OOM
        tokenized = tokenizer(
            examples["text"],
            truncation=True,
            max_length=160,
            padding="max_length",
            return_tensors="pt",
            add_special_tokens=True
        )
        # Convert to lists and immediately free GPU memory
        result = {
            "input_ids": tokenized["input_ids"].cpu().tolist(),
            "attention_mask": tokenized["attention_mask"].cpu().tolist(),
            "labels": tokenized["input_ids"].cpu().tolist()
        }
        del tokenized  # Explicitly delete to free memory
        torch.cuda.empty_cache() if torch.cuda.is_available() else None
        return result
        
    except RuntimeError as e:
        if "out of memory" in str(e).lower():
            print("⚠️ OOM during tokenization. Using fallback")
            return empty_tokenization(160)
        elif "Numpy is not available" in str(e):
            print("⚠️ Numpy error. Using fallback")
            return empty_tokenization(160)
        raise

def empty_tokenization(length):
    """Create empty tokenization to prevent crashes"""
    return {
        "input_ids": [[0]*length],
        "attention_mask": [[1]*length],
        "labels": [[0]*length]
    }

# 3. Main processing function with memory profiling
@profile
def process_dataset():
    try:
        print("\n=== Starting Processing ===")
        dataset = prepare_dataset()
        
        # Small batch test first
        test_batch = dataset.select(range(2))
        test_tokenized = test_batch.map(safe_tokenize, batched=True)
        
        # Full processing with memory optimizations
        tokenized_dataset = dataset.map(
            safe_tokenize,
            batched=True,
            batch_size=8,  # Increased for better efficiency
            remove_columns=dataset.column_names,  # Critical for memory savings
            load_from_cache_file=False  # Prevents cache buildup
        )
        
        tokenized_dataset.set_format(type='torch')
        print(f"✅ Processing completed! Final size: {len(tokenized_dataset)} samples")
        return tokenized_dataset
        
    except Exception as e:
        print(f"\n❌ Critical Error: {str(e)}")
        print("Creating minimal fallback dataset...")
        return Dataset.from_dict({
            "input_ids": [torch.tensor([0]*160)],
            "attention_mask": [torch.tensor([1]*160)],
            "labels": [torch.tensor([0]*160)]
        })

# 4. Execute processing
final_dataset = process_dataset()

# 5. Verify output
print("\nDataset sample:", final_dataset[0])
print("Memory profiling complete!")

ERROR: Could not find file /tmp/ipykernel_239/496354247.py

=== Starting Processing ===

❌ Dataset preparation failed: No JSON files found in directory: /kaggle/input/database4
Creating minimal fallback dataset...


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

✅ Processing completed! Final size: 10 samples

Dataset sample: {'input_ids': tensor([36674,  2420,   657, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
        50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 

In [25]:
# Cell 6: Training Execution
# =========================

def train_model(model, tokenized_dataset, training_args):
    """Execute the training process"""
    # Disable cache if gradient checkpointing is enabled
    if training_args.gradient_checkpointing:
        model.config.use_cache = False
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
        data_collator=data_collator
    )
    
    # 2. Start training
    print("\n=== Starting Training ===")
    trainer.train()
    
    # 3. Save model
    output_dir = f"./{MODEL_NAME.split('/')[-1]}-crypto-expert"
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"\n✅ Model saved to {output_dir}")

trainer = train_model(model, tokenized_dataset, training_args)

RecursionError: maximum recursion depth exceeded

In [ ]:
# Cell 7: Enhanced Model Saving with Shard Support
# ===============================================

def save_model_artifacts(
    model, 
    tokenizer, 
    training_args: Optional[TrainingArguments] = None, 
    output_dir: str = "/kaggle/working/gpt2-lora-trained"
) -> str:
    """
    Save all model artifacts with comprehensive verification.
    Handles both single-file and sharded model formats.
    """
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    print(f"\n💾 Saving model artifacts to: {output_dir}")
    
    # For LoRA models - DON'T merge adapters before saving
    # We want to save the adapter separately
    print("💽 Saving model and adapter...")
    
    # Save the entire model (base model + adapter)
    model.save_pretrained(
        output_dir,
        safe_serialization=True,
        state_dict=model.state_dict()  # Save the complete state including LoRA
    )
    
    # Save tokenizer
    print("🔤 Saving tokenizer...")
    tokenizer.save_pretrained(output_dir)
    
    # Save training arguments if provided
    if training_args is not None:
        print("📝 Saving training arguments...")
        try:
            args_path = os.path.join(output_dir, "training_args.json")
            if hasattr(training_args, 'to_dict'):
                with open(args_path, "w") as f:
                    json.dump(training_args.to_dict(), f, indent=2)
            elif hasattr(training_args, 'to_json_string'):
                with open(args_path, "w") as f:
                    f.write(training_args.to_json_string())
            else:
                print("⚠️ Warning: TrainingArguments has no serialization method")
        except Exception as e:
            print(f"⚠️ Warning: Failed to save training args - {str(e)}")
    
    # Verify the adapter files were saved
    required_files = ['adapter_config.json', 'adapter_model.safetensors']
    missing_files = []
    for file in required_files:
        if not os.path.exists(os.path.join(output_dir, file)):
            missing_files.append(file)
    
    if missing_files:
        print(f"\n⚠️ Warning: Missing adapter files: {missing_files}")
        print("Trying alternative save method...")
        # Explicitly save the adapter
        model.save_pretrained(
            output_dir,
            safe_serialization=True,
            adapter_only=True  # This ensures adapter files are saved
        )
    
    print("\n🔍 Verifying saved files:")
    for file in os.listdir(output_dir):
        size = os.path.getsize(os.path.join(output_dir, file)) / 1024
        print(f"- {file} ({size:.2f} KB)")
    
    return output_dir

In [ ]:
# Cell 8: Robust Model Loading and Testing with PEFT support
# ========================================================
def load_and_test_model(
    model_path: str = "/kaggle/working/gpt2-lora-trained", 
    max_length: int = 160,
    test_prompts: Optional[list] = None,
    is_peft_model: bool = True
):
    """
    Load and test a saved model with comprehensive error handling
    """
    print(f"\n🔍 Preparing to load model from: {model_path}")
    
    # Verify model directory exists
    if not os.path.exists(model_path):
        raise ValueError(f"Model directory {model_path} does not exist")
    
    # Show directory contents for debugging
    print("\n📂 Model directory contents:")
    for f in sorted(os.listdir(model_path)):
        size = os.path.getsize(os.path.join(model_path, f)) / 1024
        print(f"- {f} ({size:.2f} KB)")
    
    try:
        print("\n🔄 Loading tokenizer...")

        tokenizer = AutoTokenizer.from_pretrained(
            model_path,
            local_files_only=True
        )
        
        print("\n🔄 Loading model...")
        if is_peft_model:
            # First check if we have adapter files
            adapter_files = [
                f for f in os.listdir(model_path) 
                if f.startswith('adapter_') or f == 'adapter_config.json'
            ]
            
            if not adapter_files:
                print("⚠️ No adapter files found. Loading as regular model.")
                model = AutoModelForCausalLM.from_pretrained(
                    model_path,
                    device_map="auto",
                    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                    local_files_only=True
                )
            else:
                print(f"Found adapter files: {adapter_files}")
                # Load base model first
                base_model = AutoModelForCausalLM.from_pretrained(
                    model_path,
                    device_map="auto",
                    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                    local_files_only=True
                )
                
                # Then load the PEFT adapter
                model = PeftModel.from_pretrained(
                    base_model,
                    model_path,
                    local_files_only=True
                )
                
                # Merge and unload for inference
                model = model.merge_and_unload()
        else:
            # For regular models
            model = AutoModelForCausalLM.from_pretrained(
                model_path,
                device_map="auto",
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                local_files_only=True
            )
            
        print("\n🎉 Model loaded successfully!")
        
        # Default test prompts if none provided
        if test_prompts is None:
            test_prompts = [
                "What is hardware wallet?? ",
                "What is Proof of Work (PoW)?? ",
                "What is cryptography?? ",
                "What is Peer-to-Peer (P2P)?? ",
                "What is block chain?? ",
                "What is private key?? "
            ]
        
        # Create pipeline - REMOVED device parameter since we're using device_map="auto"
        print("\n🚀 Creating text generation pipeline...")
        pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
        )
        
        # Run tests
        print("\n🧪 Running generation tests...")
        for i, prompt in enumerate(test_prompts, 1):
            print(f"\n🔹 Test {i}: {prompt}")
            output = pipe(
                prompt,
                max_length=max_length,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                num_return_sequences=1,
                repetition_penalty=1.2
            )
            print("💬 Response:", output[0]['generated_text'])
            
        return model, tokenizer
        
    except Exception as e:
        print(f"\n❌ Critical error loading model: {str(e)}")
        print("\n🛠️ Debugging info:")
        print(f"- Path: {os.path.abspath(model_path)}")
        print(f"- Directory exists: {os.path.exists(model_path)}")
        if os.path.exists(model_path):
            print("- Contents:", os.listdir(model_path))
        raise

In [ ]:
# Main execution
if __name__ == "__main__":
    model_path = "/kaggle/working/gpt2-lora-trained"
    
    # Save model artifacts
    save_model_artifacts(model, tokenizer, training_args)
    
    # Load with explicit path and PEFT flag
    load_and_test_model(model_path, is_peft_model=True)
    
    # Test with custom prompts
    custom_prompts = [
        "What is software wallet, and what's the difference between hardware and software wallet? ",
        "What is PoW? ",
        "Explain PoW in 1 sentence. ",
        "Describe the key features of PoW using 3 words. ",
        "What is PoM? Is it something related to cryptography? ",
        "What is a cryptographic product? ",
        "What is P2P? ",
        "What is block chain? ",
        "What is public key, and what's the difference between private and public key? "
    ]
    load_and_test_model(model_path, test_prompts=custom_prompts, is_peft_model=True)

In [ ]:

notebook_end = time.time()
print(f"Total notebook execution time: {notebook_end - notebook_start:.2f} seconds")